In [26]:
import pandas as pd
from surprise import Dataset
from surprise import Reader
import joblib
import kagglehub
import joblib
from surprise import KNNWithMeans
from surprise.model_selection import cross_validate

In [27]:
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")
df = pd.read_csv(f"{path}/ratings.csv")

In [28]:
df = df.drop(labels=['timestamp'], axis=1)
df = df.dropna()

In [29]:
df.sort_values(by="movieId")
value_counts = df['movieId'].value_counts()
df = df[df['movieId'].isin(value_counts[value_counts > 2].index)]

df = df.loc[1:1000000]
# df['movieId'].value_counts()
df.shape

(999294, 3)

In [30]:
ratings_dict = {
    "item": df['movieId'],
    "user": df['userId'],
    "rating": df['rating'],
}

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(0, 5))

data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)

In [31]:
sim_options = {
    "name": "pearson_baseline",
    "user_based": False,  # Compute similarities between items
}
model = KNNWithMeans(sim_options=sim_options)

In [32]:
cv = cross_validate(model, data, measures = ['RMSE', 'MAE'], cv = 5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8265  0.8235  0.8230  0.8242  0.8262  0.8247  0.0014  
MAE (testset)     0.6239  0.6221  0.6216  0.6224  0.6239  0.6228  0.0010  
Fit time          14.03   14.59   14.79   14.61   14.73   14.55   0.27    
Test time         20.16   20.88   21.05   21.09   20

In [35]:
train_set = data.build_full_trainset()
out = model.fit(train_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


array([3.46335079, 4.33393747, 4.2778169 , ..., 2.        , 4.        ,
       3.5       ])